In [11]:
import pandas as pd 
import os
os.getcwd()
#data = pd.read_csv("dataset_HW3.csv")
os.chdir('/Users/simonlegendre/Desktop/DATSMA2/AlgorithmsInDataScience/Projet1/Code/P3') 
data = pd.read_csv("resources/dataset_HW3.csv")


,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,female,12/1950,26904,High School,Retired,2,married,Africa,2,0.0,Own house,Alzheimer's disease
1,Lillian Cole,female,4/1940,78088,PhD/md,Retired,1,married,Africa,1,0.0,Rent flat,multiple sclerosis
2,Lillian Simpson,female,12/1966,51502,Bachelor,Employed,1,married,Asia,0,0.1,Rent flat,heart disease
3,Avery Richards,female,4/1943,54080,Bachelor,Retired,1,married,Africa,0,0.0,Rent room,endometriosis
4,Sophia Alvarado,female,10/1947,68785,High School,Retired,1,married,Europe,0,0.0,Rent flat,gastritis
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Connor Pena,male,11/1997,71300,High School,Student,0,divorced,Europe,0,1.1,Rent house,gastritis
1996,Caleb Bowman,male,11/1999,49030,High School,Student,0,single,North America,0,0.1,Rent house,gastritis
1997,Ella Barber,female,6/1948,92883,Masters,Retired,5,married,Africa,1,0.0,Own house,skin cancer
1998,Gavin Wilson,male,4/1976,42200,High School,Employed,1,married,Asia,2,0.0,Own flat,diabetes
